# How to build a simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to store memory in a json file.


A chatbot app is a software application designed to simulate conversation with human users. It uses artificial intelligence (AI) to understand what someone is saying and to respond with appropriate answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-chatbot.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-simple-chatbot**.

## Trick to avoid the nasty deprecation warnings from LangChain

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [5]:
from langchain_groq import ChatGroq

chatbot = ChatGroq(
            groq_api_key=groq_api_key,
            model = "llama3-70b-8192", 
            temperature = 1.0,)

* Human Message: the user input.

In [7]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

#### Call the ChatModel (the LLM)

In [8]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content='Blue is a calming and peaceful color that represents tranquility and serenity. It is often associated with the sky and the ocean, evoking feelings of freedom and openness. Blue is also known to symbolize trust, loyalty, and wisdom. Overall, blue is a versatile and timeless color that is loved by many.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 13, 'total_tokens': 76}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c04030ec-cc97-4525-bcb6-617d9b705b64-0', usage_metadata={'input_tokens': 13, 'output_tokens': 63, 'total_tokens': 76})

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Check if the Chatbot remembers your favorite color.

In [9]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I do not have access to that information.", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a31d0a71-a438-4cf2-aae1-8b992b0581a7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [7]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.93
    Uninstalling langsmith-0.1.93:
      Successfully uninstalled langsmith-0.1.93
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.22
    Uninstalling langchain-core-0.2.22:
      Successfully uninstalled langchain-core-0.2.22
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.2
    Uninstalling langchain-text-splitters-0.2.2:
      Successfully uninstalled langchain-text-splitters-0.2.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.10
    Uninstalling langchain-0.2.10:
      Successfully uninstalled langchain-0.2.10
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

In [8]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [9]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("new_messages.json"),
    memory_key="messages",
    return_messages=True
)

In [10]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

## che minchia fa quel prompt template?

In [11]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.invoke("HI i am andrea!")

{'content': 'HI i am andrea!',
 'messages': [],
 'text': "Hello Andrea! It's nice to meet you! How's your day going so far?"}

Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')
Failed to batch ingest runs: LangSmithError('Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError(\'403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Forbidden"}\')')


In [13]:
chain.invoke("Do you remember what i said?")

{'content': 'Do you remember what i said?',
 'messages': [HumanMessage(content='HI i am andrea!'),
  AIMessage(content="Hello Andrea! It's nice to meet you! How's your day going so far?")],
 'text': "You introduced yourself as Andrea. That's about it so far in our conversation!"}

* Check the file messages.json in the root directory.
* This is just a simple example, in the real world you probably will not save your memory in a json file. 
* And remember: the context window is limited and it affects to the cost of using chatGPT API.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-chatbot.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-chatbot.py